In [59]:
import pandas as pd

In [60]:
source_df = pd.read_csv("Forwards.csv")

In [61]:
source_df.head(2)

,India Id,Trade Id,Product Description,Trade Date,Start Date,MR_MATURITY_DATE,Entered Date,CounterParty (F),Book,Buy/Sell,Currency Pair,Prim Cur,Prim Amt,Trade Price,Sec Cur,Sec Amt,TradeStatus
0,26997,13782015.0,2199667/FXFlexiForward/MerchantFX/RETAIL/USD/I...,01-03-21 0:00,01-09-21,07-03-24,01-03-21 0:00,Counterparty 1,TB.IND.TRADING.TIMEOPT,Buy,USD/INR,USD,450000,75.38,INR,-33921000.00,VERIFIED
1,27966,14734932.0,2219298/FXFlexiForward/MerchantFX/RETAIL/USD/I...,12-03-21 0:00,30-07-21,23-03-25,12-03-21 0:00,Counterparty 2,TB.IND.TRADING.TIMEOPT,Buy,USD/INR,USD,83395.63,74.21,INR,-6188581.22,VERIFIED


In [62]:
import json
with open('sample.json') as json_file:
    quality_check_obj = json.load(json_file)

    # Print the type of data variable
    print("Type:", type(quality_check_obj))
    
print(quality_check_obj)

Type: <class 'dict'>
{'missing_columns_source': [], 'is_pkduplicated': True, 'is_pknull': True, 'primary_key_dict': {'India Id': {'duplicated': [], 'null': []}, 'Trade Id': {'duplicated': [1, 8, 13, 24, 26], 'null': [14]}, 'Product Description': {'duplicated': [4, 20], 'null': [4, 20]}, 'CounterParty (F)': {'duplicated': [], 'null': []}}, 'datatype_missmatch': {'India Id': {'both_same': True, 'datatypes': {'identified_data_type': 'integer', 'all_same_datatype': True, 'null_index': [], 'integer_details': {'int_index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26], 'int_nagative_index': [], 'int_zero_index': []}, 'string_index': [], 'decimal_details': {'decimal_index': [], 'decimal_format': ''}, 'date_details': {'date_index': {}, 'date_format': []}}}, 'Trade Id': {'both_same': False, 'datatypes': {'identified_data_type': 'integer', 'all_same_datatype': False, 'null_index': [14], 'integer_details': {'int_index': [0, 1, 2, 3, 4, 5, 6, 7,

In [63]:

def create_checks_excel(source_df, quality_check_obj):
    pd.options.mode.chained_assignment = None
    
    copy_source = source_df
    missing_columns = quality_check_obj["missing_columns_source"]
    
    # if not quality_checks_dict["MissingColumns"]["discard"]: #include
    #     for col in missing_columns:
    #         copy_source[col +("(missing)")] = ""
    # else: # dicard columns
    #     copy_source.drop(missing_columns)

    copy_source["check_fail_errors"] = ""
    copy_source["discard_flag"] = 0 # 0 -keep 1-discard
    
        
    ## adding primarykey and duplicated missing check_fail_errors...    
    if quality_check_obj["is_pkduplicated"] and quality_check_obj["is_pknull"]: 
        pk_dict = quality_check_obj["primary_key_dict"]           
        for col in pk_dict:            
            if len(pk_dict[col]["null"]) > 0 : # index of null values
                for i in pk_dict[col]["null"]:
                    copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + ": Primary key is Null ]"
                    
            if len(pk_dict[col]["duplicated"]) > 0 : # index of duplicated value
                for i in pk_dict[col]["duplicated"]: # i is cell_index
                    copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + ": Primary key is duplicated ]"
    
    ## adding datatype missmatch check_fail_errors
    dt_dict = quality_check_obj["datatype_missmatch"]
    
    set2 = set(i for i in range(0, source_df.shape[0])) # all rows index
    
    for col in dt_dict:
        if not dt_dict[col]["both_same"]: # if source and report columns not same
            
            datatype_obj = dt_dict[col]["datatypes"]
            
            if len(datatype_obj["null_index"]): # any datatype null should be highlighted
                for i in datatype_obj["null_index"]:
                    copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " Null value ]"
            
            if not datatype_obj["all_same_datatype"]: # if not same than hightlight it
                
                if datatype_obj["identified_data_type"] == "integer": # majority int found                    
                    
                    int_details = datatype_obj["integer_details"]
                    # all integer include zero and -ve
                    set_int = set(int_details["int_index"]) #int list which not need to hightlight
                    not_int_set = set2 - set_int # index which hv not int
                    for i in not_int_set:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not Integer ]"
                    
                    # all integer not zero
                    set_zero = set(int_details["int_zero_index"]) #int list which is 0
                    for i in set_zero:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " is Zero ]"
                        
                    # all integer not -ve
                    set_negative = set(int_details["int_nagative_index"]) #int list which not need to hightlight
                    for i in set_negative:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " is Negative ]"
                        
                elif datatype_obj["identified_data_type"] == "string": # majority string found
                    set_string = set(datatype_obj["string_index"]) #string list which not need to hightlight
                    not_string_set = set2 - set_string # index which hv not string..
                    
                    for i in not_string_set:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not String ]"
                        
                elif "decimal" in datatype_obj["identified_data_type"]:# majority decimal found
                    decimal_detail = datatype_obj["decimal_details"]
                    set_decimal = set(decimal_detail["decimal_index"]) #decimal list which not need to hightlight
                    diff = set2 - set_decimal # index which hv not decimal
                    for i in diff:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not decimal with " + decimal_detail["decimal_format"] + " format]"
                
                elif "Date" in datatype_obj["identified_data_type"]:
                    date_detail = datatype_obj["date_details"]                    
                    set_date = set(date_detail["date_index"])
                    
                    diff = set2 - set_date # index which hv not decimal
                    for i in diff:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not a date format with " + str(date_detail["date_format"]) + " format]"                             
                             
    print("*********** new excel witch check file records *******")  
    print(copy_source.head(2))
    
    copy_source.to_csv('sample.csv')
    
    
    return True
 

In [64]:
created = create_checks_excel(source_df, quality_check_obj)

*********** new excel witch check file records *******
   India Id    Trade Id                                Product Description  \
0     26997  13782015.0  2199667/FXFlexiForward/MerchantFX/RETAIL/USD/I...   
1     27966  14734932.0  2219298/FXFlexiForward/MerchantFX/RETAIL/USD/I...   

      Trade Date Start Date MR_MATURITY_DATE   Entered Date CounterParty (F)  \
0  01-03-21 0:00   01-09-21         07-03-24  01-03-21 0:00   Counterparty 1   
1  12-03-21 0:00   30-07-21         23-03-25  12-03-21 0:00   Counterparty 2   

                     Book Buy/Sell Currency Pair Prim Cur  Prim Amt  \
0  TB.IND.TRADING.TIMEOPT      Buy       USD/INR      USD    450000   
1  TB.IND.TRADING.TIMEOPT      Buy       USD/INR      USD  83395.63   

  Trade Price Sec Cur      Sec Amt TradeStatus  \
0       75.38     INR -33921000.00    VERIFIED   
1       74.21     INR  -6188581.22    VERIFIED   

                                   check_fail_errors  discard_flag  
0  [Trade Date not a date format wi

In [65]:
import openpyxl

In [66]:

def create_checks_excel_summary(source_df, quality_check_obj):
    pd.options.mode.chained_assignment = None
    
    # copy_source = source_df
    missing_columns = quality_check_obj["missing_columns_source"]
    
    summary_df = pd.DataFrame()
    
    copy_source["check_fail_errors"] = ""
    copy_source["discard_flag"] = 0 # 0 -keep 1-discard
    
        
    ## adding primarykey and duplicated missing check_fail_errors...    
    if quality_check_obj["is_pkduplicated"] and quality_check_obj["is_pknull"]: 
        pk_dict = quality_check_obj["primary_key_dict"]           
        for col in pk_dict:            
            if len(pk_dict[col]["null"]) > 0 : # index of null values
                for i in pk_dict[col]["null"]:
                    copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + ": Primary key is Null ]"
                    
            if len(pk_dict[col]["duplicated"]) > 0 : # index of duplicated value
                for i in pk_dict[col]["duplicated"]: # i is cell_index
                    copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + ": Primary key is duplicated ]"
    
    ## adding datatype missmatch check_fail_errors
    dt_dict = quality_check_obj["datatype_missmatch"]
    
    set2 = set(i for i in range(0, source_df.shape[0])) # all rows index
    
    for col in dt_dict:
        if not dt_dict[col]["both_same"]: # if source and report columns not same
            
            datatype_obj = dt_dict[col]["datatypes"]
            
            if len(datatype_obj["null_index"]): # any datatype null should be highlighted
                for i in datatype_obj["null_index"]:
                    copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " Null value ]"
            
            if not datatype_obj["all_same_datatype"]: # if not same than hightlight it
                
                if datatype_obj["identified_data_type"] == "integer": # majority int found                    
                    
                    int_details = datatype_obj["integer_details"]
                    # all integer include zero and -ve
                    set_int = set(int_details["int_index"]) #int list which not need to hightlight
                    not_int_set = set2 - set_int # index which hv not int
                    for i in not_int_set:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not Integer ]"
                    
                    # all integer not zero
                    set_zero = set(int_details["int_zero_index"]) #int list which is 0
                    for i in set_zero:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " is Zero ]"
                        
                    # all integer not -ve
                    set_negative = set(int_details["int_nagative_index"]) #int list which not need to hightlight
                    for i in set_negative:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " is Negative ]"
                        
                elif datatype_obj["identified_data_type"] == "string": # majority string found
                    set_string = set(datatype_obj["string_index"]) #string list which not need to hightlight
                    not_string_set = set2 - set_string # index which hv not string..
                    
                    for i in not_string_set:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not String ]"
                        
                elif "decimal" in datatype_obj["identified_data_type"]:# majority decimal found
                    decimal_detail = datatype_obj["decimal_details"]
                    set_decimal = set(decimal_detail["decimal_index"]) #decimal list which not need to hightlight
                    diff = set2 - set_decimal # index which hv not decimal
                    for i in diff:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not decimal with " + decimal_detail["decimal_format"] + " format]"
                
                elif "Date" in datatype_obj["identified_data_type"]:
                    date_detail = datatype_obj["date_details"]                    
                    set_date = set(date_detail["date_index"])
                    
                    diff = set2 - set_date # index which hv not decimal
                    for i in diff:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not a date format with " + str(date_detail["date_format"]) + " format]"                             
                             
    print("*********** new excel witch check file records *******")  
    print(copy_source.head(2))
    
    copy_source.to_csv('sample.csv')
    
    
    return True
 

In [93]:
from openpyxl.utils.dataframe import dataframe_to_rows

def value_to_excel(v):
    return str(v).replace("[]","")

workbook = openpyxl.Workbook()

source_ws = workbook.active
source_ws.title = 'source_df'


rows = dataframe_to_rows(source_df, index=False)
for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
        source_ws.cell(row=r_idx, column=c_idx, value=value)
        
        
summary_ws = workbook.create_sheet('summary')
# Construct test worksheets with some content.
rows = [
    ["Missing Source Column", value_to_excel(quality_check_obj["missing_columns_source"]) ],
    ["Primary Key Duplicated", value_to_excel(quality_check_obj["is_pkduplicated"])],
    ["Primary Has Null Values", value_to_excel(quality_check_obj["is_pknull"])],
    ["Primary Key Details"]
    ]

for col, values in quality_check_obj["primary_key_dict"].items():
    v = str(values).replace("[]","")
    # print(col)
    # print(v)
    rows.append([col, v])
    
for col, values in quality_check_obj["datatype_missmatch"].items():
    v = str(values).replace("[]","")
    rows.append([col, v])
# print(rows)
for row in rows:
    # print(row)
    summary_ws.append(row)
    
workbook.save('quality_checks.xlsx')

In [ ]:

def create_quality_checks_xlxs_ons3(source_df, quality_check_obj):    
    
    pd.options.mode.chained_assignment = None
    
    copy_source = source_df
    missing_columns = quality_check_obj.missing_columns_source
    
    if not quality_checks_dict["MissingColumns"]["discard"]: #include
        for col in missing_columns:
            copy_source[col +("(missing)")] = ""
    else: # dicard columns
        copy_source.drop(missing_columns)

    copy_source["check_fail_errors"] = ""
    copy_source["discard_flag"] = 0 # 0 -keep 1-discard
    
        
    ## adding primarykey and duplicated missing check_fail_errors...    
    if quality_check_obj.is_pkduplicated and quality_check_obj.is_pknull: 
        pk_dict = quality_check_obj.primary_key_dict           
        for col in pk_dict:            
            if len(pk_dict[col]["null"]) > 0 : # index of null values
                for i in pk_dict[col]["null"]:
                    copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + ": Primary key is Null ]"
                    
            if len(pk_dict[col]["duplicated"]) > 0 : # index of duplicated value
                for i in pk_dict[col]["duplicated"]: # i is cell_index
                    copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + ": Primary key is duplicated ]"
    
    ## adding datatype missmatch check_fail_errors
    dt_dict = pydantic_to_dict(quality_check_obj.datatype_missmatch)
    
    set2 = set(i for i in range(0, source_df.shape[0])) # all rows index
    
    for col in dt_dict:
        if not dt_dict[col]["both_same"]: # if source and report columns not same
            
            datatype_obj = DataTypes( **dt_dict[col]["datatypes"])
            
            if len(datatype_obj.null_index): # any datatype null should be highlighted
                for i in datatype_obj.null_index:
                    copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " Null value ]"
            
            if not datatype_obj.all_same_datatype: # if not same than hightlight it
                
                if datatype_obj.identified_data_type == "integer": # majority int found                    
                    
                    int_details = datatype_obj.integer_details
                    # all integer include zero and -ve
                    set_int = set(int_details.int_index) #int list which not need to hightlight
                    not_int_set = set2 - set_int # index which hv not int
                    for i in not_int_set:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not Integer ]"
                    
                    # all integer not zero
                    set_zero = set(int_details.int_zero_index) #int list which is 0
                    for i in set_zero:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " is Zero ]"
                        
                    # all integer not -ve
                    set_negative = set(int_details.int_nagative_index) #int list which not need to hightlight
                    for i in set_negative:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " is Negative ]"
                        
                elif datatype_obj.identified_data_type == "string": # majority string found
                    set_string = set(datatype_obj.string_index) #string list which not need to hightlight
                    not_string_set = set2 - set_string # index which hv not string..
                    
                    for i in not_string_set:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not String ]"
                        
                elif "decimal" in datatype_obj.identified_data_type:# majority decimal found
                    decimal_detail = datatype_obj.decimal_details
                    set_decimal = set(decimal_detail.decimal_index) #decimal list which not need to hightlight
                    diff = set2 - set_decimal # index which hv not decimal
                    for i in diff:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not decimal with " + decimal_detail.decimal_format + " format]"
                
                elif "Date" in datatype_obj.identified_data_type:
                    date_detail = datatype_obj.date_details                    
                    set_date = set(date_detail.date_index)
                    
                    diff = set2 - set_date # index which hv not decimal
                    for i in diff:
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not a date format with " + str(date_detail.date_format) + " format]"                             
                             
    print("*********** new excel witch check file records *******")  
    print(copy_source.head(2))
    
    copy_source.to_csv('sample.csv')
    
    
    return True



In [ ]:

def create_quality_checks_xlxs_ons3(source_df, quality_check_obj):    
    
    def value_to_excel(v):
        return str(v).replace("[]","")

    workbook = openpyxl.Workbook()
    source_ws = workbook.active
    source_ws.title = 'source'
    summary_ws = workbook.create_sheet('summary')
    

    ## creating extra columns
    copy_source = source_df
    missing_columns = quality_check_obj.missing_columns_source
    
    if not quality_checks_dict["MissingColumns"]["discard"]: #include
        for col in missing_columns:
            copy_source[col +("(missing)")] = ""
    else: # dicard columns
        copy_source.drop(missing_columns)

    copy_source["check_fail_errors"] = ""
    copy_source["discard_flag"] = 0 # 0 -keep 1-discard
    
    summary_rows = [
    ["Missing Source Column", value_to_excel(quality_check_obj["missing_columns_source"]) ],
    ["Primary Key Duplicated", value_to_excel(quality_check_obj["is_pkduplicated"])],
    ["Primary Has Null Values", value_to_excel(quality_check_obj["is_pknull"])]
    ]
            
    ## adding primarykey and duplicated missing check_fail_errors...    
    if quality_check_obj.is_pkduplicated and quality_check_obj.is_pknull: 
        pk_dict = quality_check_obj.primary_key_dict
                  
        for col in pk_dict:
            if len(pk_dict[col]["null"]) > 0: #if index is not null
                summary_rows.append(["Number of Null in Primary Keys"])             
                summary_rows.append([col, len(pk_dict[col]["null"])]) # colname: len - for xl summary                 
                    
                for i in pk_dict[col]["null"]:
                    if not quality_checks_dict["PrimaryKeyDuplicates"]["discard"]:# not discard, notify...
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + ": Primary key is Null ]"
                    else:
                        copy_source["discard_flag"][i] = 1

            if len(pk_dict[col]["duplicated"]) > 0: # index of duplicated value                 
                summary_rows.append(["Number of Duplicated in Primary Keys"])             
                summary_rows.append([col, len(pk_dict[col]["duplicated"])]) # colname: len - for xl summary
                
                for i in pk_dict[col]["duplicated"]: # i is cell_index
                    if not quality_checks_dict["PrimaryKeyDuplicates"]["discard"]:#not discard, notify
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + ": Primary key is duplicated ]"
                    else:
                        copy_source["discard_flag"][i] = 1
    
    ## adding datatype missmatch check_fail_errors
    dt_dict = pydantic_to_dict(quality_check_obj.datatype_missmatch)
    
    set2 = set(i for i in range(0, source_df.shape[0])) # all rows index
    
    for col in dt_dict:
        if not dt_dict[col]["both_same"]: # if source and report columns not same
            
            datatype_obj = DataTypes( **dt_dict[col]["datatypes"])
            
            if len(datatype_obj.null_index) > 0: # any datatype null should be highlighted
                for i in datatype_obj.null_index:
                    if not quality_checks_dict["DataTypeMismatch"]["discard"]: # if not discard, notify
                        copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " Null value ]"
                    else:
                        copy_source["discard_flag"][i] = 1
            
            if not datatype_obj.all_same_datatype: # if not same than hightlight it
                
                if datatype_obj.identified_data_type == "integer": # majority int found                    
                    
                    int_details = datatype_obj.integer_details
                    # all integer include zero and -ve
                    set_int = set(int_details.int_index) #int list which not need to hightlight
                    not_int_set = set2 - set_int # index which hv not int
                    for i in not_int_set:
                        if not quality_checks_dict["DataTypeMismatch"]["discard"]: # if not discard, notify
                            copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not Integer ]"
                        else:
                            copy_source["discard_flag"][i] = 1
                            
                    # all integer not zero
                    set_zero = set(int_details.int_zero_index) #int list which is 0
                    for i in set_zero:
                        if not quality_checks_dict["DataTypeMismatch"]["discard"]: # if not discard, notify
                            copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " is Zero ]"
                        else:
                            copy_source["discard_flag"][i] = 1
                        
                    # all integer not -ve
                    set_negative = set(int_details.int_nagative_index) #int list which not need to hightlight
                    for i in set_negative:
                        if not quality_checks_dict["DataTypeMismatch"]["discard"]: #if not discard, notify
                            copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " is Negative ]"
                        else:
                            copy_source["discard_flag"][i] = 1
                            
                elif datatype_obj.identified_data_type == "string": # majority string found
                    set_string = set(datatype_obj.string_index) #string list which not need to hightlight
                    not_string_set = set2 - set_string # index which hv not string..
                    
                    for i in not_string_set:
                        if not quality_checks_dict["DataTypeMismatch"]["discard"]: 
                            copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not String ]"
                        else:
                            copy_source["discard_flag"][i] = 1
                            
                elif "decimal" in datatype_obj.identified_data_type:# majority decimal found
                    decimal_detail = datatype_obj.decimal_details
                    set_decimal = set(decimal_detail.decimal_index) #decimal list which not need to hightlight
                    diff = set2 - set_decimal # index which hv not decimal
                    for i in diff:
                        if not quality_checks_dict["DataTypeMismatch"]["discard"]:
                            copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not decimal with " + decimal_detail.decimal_format + " format]"
                        else:
                            copy_source["discard_flag"][i] = 1
                            
                elif "Date" in datatype_obj.identified_data_type:
                    date_detail = datatype_obj.date_details                    
                    set_date = set(date_detail.date_index)
                    
                    diff = set2 - set_date # index which hv not decimal
                    for i in diff:
                        if not quality_checks_dict["DataTypeMismatch"]["discard"]:
                            copy_source["check_fail_errors"][i] = copy_source["check_fail_errors"][i] + "[" + col + " not a date format with " + str(date_detail.date_format) + " format]"                             
                        else:
                            copy_source["discard_flag"][i] = 1
    
    ## discard data having flag 1
    copy_source = copy_source.loc[copy_source["discard_flag"] == 0]
    
    ## write source dataframes into xlxs file
    df_rows = dataframe_to_rows(copy_source, index=False)
    for r_idx, row in enumerate(df_rows, 1):
        for c_idx, value in enumerate(row, 1):
            source_ws.cell(row=r_idx, column=c_idx, value=value)
            
    for row in summary_rows:
        summary_ws.append(row)
    
    workbook.save('quality_checks_summary.xlsx')                              
                            
    print("*********** created quality checks with summary.. *******")  
    
    return True



In [96]:

int_value = int("178t")
# str(int_value) == s
    

ValueError: invalid literal for int() with base 10: '178t'

In [111]:
s = 'Date (dd-mm-yyyy hh:mm)'
s = s.replace("Date (","").replace(")","")

print(s)


dd-mm-yyyy hh:mm


In [118]:
from datetime import datetime

s = '01-03-2021 1:00'
datetime.strptime(s, '%d-%m-%Y %H:%M')


datetime.datetime(2021, 3, 1, 1, 0)

In [120]:
not("decimal" in "decimal(10,2)")

False

In [131]:
ff = float("3")

int_f = int(ff)
print(str(int_f))

3


In [132]:
str(ff)

'3.0'

In [133]:
str(int_f)

'3'

In [ ]:

def get_data_type_v2(sr):
    def is_none_nan_or_blank(var):
        if var is None:
            return True

        if isinstance(var, float) and math.isnan(var):
            return True

        if isinstance(var, str) and var.strip() == '':
            return True

        return False

    def get_date_format(date_string):
        format_list = []
        for dt_format in format_dict:
            try:
                datetime.strptime(date_string, format_dict[dt_format])
                format_list.append("Date (" + dt_format + ")")
            except ValueError:
                pass
        return format_list

    def is_integer(s):
        try:
            int_value = int(s)
            return str(int_value) == s
        except ValueError:
            return False
    
    v_decimal = []
    v_int = []
    v_int_nagative = []
    v_int_zero = []
    v_string = []
    v_date = dict()
    v_none = []
    max_after_decimal = 0
    max_before_decimal = 0
    date_format_list = []

    for i, v in sr.items():
        if is_none_nan_or_blank(v):
            v_none.append(i)
            continue

        if isinstance(v, np.generic):
            v = v.item()

        if is_integer(v):
            v_int.append(i)
            if int(v) == 0:
                v_int_zero.append(i)
            elif int(v) < 0:
                v_int_nagative.append(i)
        else:
            try:
                float(v)
                v_decimal.append(i)
            except ValueError:
                date_format = get_date_format(v)
                if date_format:
                    if i == 0:
                        v_date.update({i:date_format})
                        date_format_list.extend(date_format)
                    else:
                        for dt in date_format:
                            v_date.update({i:date_format})
                            if dt in date_format_list:
                                continue
                            else:
                                date_format_list.extend(date_format)
                        
                else:
                    v_string.append(i)

    v_decimal_len = ""
    v_decimal_format = ""
    if len(v_decimal) > 0:
        for i, v in sr.items():
            if "." in str(v):
                v = str(v).lstrip("-")
                max_after_decimal = max(max_after_decimal, len(v.split(".")[1]))
                max_before_decimal = max(max_before_decimal, len(v.split(".")[0]))
        max_after_decimal = min(10, max_after_decimal)
        v_decimal_format = f"decimal({str(max_before_decimal + max_after_decimal)},{str(max_after_decimal)})"

    ##deciding final datatype
    v_data_type = ""
    all_same_datatype = False

    if (len(v_int) > len(v_decimal) and len(v_int) > len(v_string) and len(v_int) > len(v_date)):
        v_data_type = "integer"
    elif len(v_decimal) > len(v_int) and len(v_decimal) > len(v_string) and len(v_decimal) > len(v_date):
        v_data_type = v_decimal_format
    elif len(v_date) > len(v_string):
        if len(date_format_list) == 1: # getting only one format
            v_data_type = "Date (" + str(date_format_list) + ")"
        else:
            v_data_type = "Date"
    else:
        v_data_type = "string"
    
    if ((len(v_decimal) == len(sr)) or 
        (len(v_int) == len(sr)) or
        (len(v_date) == len(sr)) or
        len(v_string) == len(sr)):
        all_same_datatype = True
    
    # return {"identified_data_type": v_data_type,
    #         "all_same_datatype": all_same_datatype, 
    #         "null_index": v_none,
    #         "integer_index": {"int_index": v_int, 
    #                         "int_nagative_index": v_int_nagative,
    #                         "int_zero_index":v_int_zero},
    #         "string_index": v_string,
    #         "decimal_details": {"decimal_index": v_decimal, 
    #                             "decimal_format": v_decimal_format },
    #         "date_details": {"date_index":v_date,
    #                          "date_format":date_format_list}            
    #         }
    
    integer_details = IntegerDetails( int_index = v_int,
                                     int_nagative_index = v_int_nagative,
                                     int_zero_index = v_int_zero)    
    decimal_details = DecimalDetails( decimal_index = v_decimal,
                                     decimal_format = v_decimal_format)    
    date_details = DateDetails( date_index = v_date,
                               date_format = date_format_list ) 
    datatype_details = DataTypes( identified_data_type = v_data_type,
                                 all_same_datatype = all_same_datatype,
                                 null_index= v_none,
                                 integer_details = integer_details,
                                 string_index = v_string,
                                 decimal_details = decimal_details,
                                 date_details = date_details)
    return datatype_details
    
    
def check_datatype_mismatch_v2(source_df, report_columns): #report_columns is b_source_column object list
    
    datatype_dict = {obj.col_name_user_defined : obj.data_type for obj in report_columns}
    
    identified_datatype_dict = dict()
    for col in source_df.columns:
        identified_datatype_dict.update({col:{"datatype_details": get_data_type(source_df[col])}}) 
               
        source_df_col_dt = identified_datatype_dict[col]["datatype_details"] ## converting to pydentic model
        
        if(source_df_col_dt.all_same_datatype):
            
            if  (("Date" in source_df_col_dt.identified_data_type) and ("Date" in datatype_dict[col]) ) or \
            (("decimal" in source_df_col_dt.identified_data_type) and ("decimal" in datatype_dict[col])) or \
            (source_df_col_dt.identified_data_type == datatype_dict[col]):    
                            
                identified_datatype_dict.update({col: DataTypesMissmatch(datatypes = source_df_col_dt, both_same= True)})
        else:
            identified_datatype_dict.update({col: DataTypesMissmatch(datatypes = source_df_col_dt, both_same= False)})
        
    return identified_datatype_dict
